# O que são arquivos de vídeo?

Arquivos de vídeo, na realidade, combinam áudio e vídeo em um único formato comprimido.

Quando temos diversas imagens passando rapidamente pelos nossos olhos, temos a impressão de movimento:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/The_Horse_in_Motion_high_res.jpg/1200px-The_Horse_in_Motion_high_res.jpg)

Se unirmos isso a uma trilha sonora, temos um vídeo!

## Taxa de quadros (frame rate)

Para uma impressão fluida de movimento, diversas imagens precisam ser reproduzidas a cada segundo.

O número de imagens a cada segundo determina a **taxa de quadros**, também conhecida por *frame rate* ou FPS (*frames per second*).

![](https://www.videoproc.com/images/vp-seo/frame-rate-fps.jpg)

Quanto maior o FPS, maior a "fluidez" do vídeo, mas também maior a quantidade de informação armazenada e processada.

No cinema, o padrão é 24 FPS.

Na televisão, esse valor pode ser de 30 FPS, 50 FPS, ou até mesmo 60 FPS (algumas TVs mais modernas inclusive fazem o "preenchimento" de frames, por isso os filmes podem parecer mais "fluídos" do que deveriam).

## Compressão e formatos

Se cada imagem do vídeo for gravada de forma independente, seria necessário um grande volume para armazenar e processar cada imagem. No lugar disso, arquivos de vídeo em geral armazenam apenas a **diferença** entre frames, o que reduz bastante a sobrecarga no processamento:

![](.https://www.researchgate.net/publication/352265804/figure/fig1/AS:11431281117608298@1675483083501/The-group-of-pictures-GOP-structure.png)

Um algoritmo capaz de fazer esse processamento (também para a trilha sonora) é chamado de **codec** (encoder-decoder). Para criarmos um vídeo em determinado formato, e também para conseguir rodá-lo depois, é preciso ter o codec específico instalado.

![](https://interactiveimmersive.io/wp-content/uploads/2023/02/image.png)

Com os codecs, os vídeos se tornam arquivos muito mais leves que as suas imagens individuais!

## Trabalhando com arquivos de vídeo

Vamos acessar um arquivo de vídeo da internet, baixá-lo e extrair seus frames para uma pasta:

https://videos.pexels.com/video-files/15921892/15921892-uhd_3840_2160_50fps.mp4

In [1]:
from pathlib import Path

path = Path('videos')
path.mkdir(parents=True, exist_ok=True)

In [3]:
import requests

url = 'https://videos.pexels.com/video-files/15921892/15921892-uhd_3840_2160_50fps.mp4'
video_data = requests.get(url, stream=True)

video_path = path / 'video.mp4'

with open(video_path, 'wb') as file:
    for chunk in video_data.iter_content(chunk_size=1024):
        file.write(chunk)

In [4]:
path_frames = path / 'frames'
path_frames.mkdir(exist_ok=True, parents=True)

In [8]:
import cv2

capture = cv2.VideoCapture(str(video_path))

success, frame = capture.read()

count = 0

fps = int(capture.get(cv2.CAP_PROP_FPS))

In [10]:
fps

50

In [11]:
while success:
    if count % fps == 0:
        frame_name = f'frame{count:03d}.jpg'
        cv2.imwrite(str(path_frames / frame_name), frame)
        print('.', end='')

    success, frame = capture.read()
    count += 1
    
print('\nDone')

................
Done


### Classificação de vídeo

Para esta tarefa, precisamos da biblioteca `decord` e da biblioteca `av`:

```bash
pip install decord av
```

Vamos usar um modelo de classificação de vídeo: https://huggingface.co/facebook/timesformer-base-finetuned-k600

In [12]:
from transformers import pipeline

model = 'facebook/timesformer-base-finetuned-k600'
video_classifier = pipeline('video-classification', model=model)

c:\Users\annaf\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\annaf\.cache\huggingface\hub\models--facebook--timesformer-base-finetuned-k600. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [14]:
from datasets import load_dataset

data = load_dataset('shinonomelab/cleanvid-15m_map', streaming=True, split='train')

In [15]:
for line in data.take(5):
    print(line)
    print('\n\n====================\n\n')

{'id': 1095327279, 'description': "Close-up of a male welder in protective clothing welding metal in a blacksmith's workshop. The process of making a metal fence in a forge.", 'duration': 7.96, 'aspectratio': '16:9', 'videourl': 'https://www.shutterstock.com/shutterstock/videos/1095327279/preview/stock-footage-close-up-of-a-male-welder-in-protective-clothing-welding-metal-in-a-blacksmith-s-workshop-the.mp4', 'author': '{"accountsId": 162697858, "contributorId": 162697858, "bio": "\\u0421\\u0432\\u0430\\u0434\\u0435\\u0431\\u043d\\u044b\\u0439 \\u0438 \\u0441\\u0435\\u043c\\u0435\\u0439\\u043d\\u044b\\u0439 \\u0432\\u0438\\u0434\\u0435\\u043e\\u0433\\u0440\\u0430\\u0444. \\u0417\\u0430\\u043d\\u0438\\u043c\\u0430\\u044e\\u0441\\u044c \\u0432\\u0438\\u0434\\u0435\\u043e\\u0441\\u044a\\u0435\\u043c\\u043a\\u043e\\u0439 \\u0441 2007 \\u0433\\u043e\\u0434\\u0430.", "location": "ua", "website": "-", "contributorTypeList": ["videographer"], "equipmentList": ["Canon 6D", "Canon 50 mm 1.4", "Ca

In [16]:
n_people = 0

for line in data:
    categories = line['categories']
    
    if 'People' not in categories:
        continue
    
    url = line['videourl']
    prediction = video_classifier(url)
    
    print(f'Video: {url}\nPredictions:')
    for p in prediction:
        print(p)
        
    print('\n\n====================\n\n')
    n_people += 1
    
    if n_people == 10:
        break

Video: https://www.shutterstock.com/shutterstock/videos/1095327279/preview/stock-footage-close-up-of-a-male-welder-in-protective-clothing-welding-metal-in-a-blacksmith-s-workshop-the.mp4
Predictions:
{'score': 0.8637503981590271, 'label': 'welding'}
{'score': 0.06536673754453659, 'label': 'using circular saw'}
{'score': 0.019854390993714333, 'label': 'making horseshoes'}
{'score': 0.015573847107589245, 'label': 'using a power drill'}
{'score': 0.009469602257013321, 'label': 'bending metal'}




Video: https://www.shutterstock.com/shutterstock/videos/1079808617/preview/stock-footage-portrait-shot-of-caucasian-woman-wearing-protective-mask-standing-outdoor-in-forest-looking-at-the.mp4
Predictions:
{'score': 0.4843772053718567, 'label': 'archery'}
{'score': 0.10088776797056198, 'label': 'stretching arm'}
{'score': 0.09427043795585632, 'label': 'huddling'}
{'score': 0.039245180785655975, 'label': 'planting trees'}
{'score': 0.03581533208489418, 'label': 'throwing axe'}




Video: https://w